In [ ]:
!pip install transformers
!pip install datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import datasets
import transformers


Mounted at /content/drive


# Start here: load dataset

In [6]:
from datasets import load_dataset
#Encoded Labels dataset
dataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/10.9.22-emo_class/emo_ds_filtered_train.csv', \
                                          'test': '/content/drive/MyDrive/10.9.22-emo_class/emo_ds_filtered_test.csv'})



  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# SECOND ATTEMPT 

from transformers import TrainingArguments, Trainer

import numpy as np
from datasets import load_metric

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer

metric = load_metric("accuracy")
 
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
def tokenize_data(example):
    return tokenizer(example['text'], padding='max_length')

dataset_trans = dataset.map(tokenize_data, batched=True)
#dataset_trans = dataset_trans.map(remove_columns='sentence')

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

training_args = TrainingArguments(output_dir="test_trainer", \
                                  evaluation_strategy="epoch", \
                                  num_train_epochs=6, \
                                  )        #(0.001) per_device_train_batch_size=16, \ learning_rate=1e-3

trainer = Trainer(
    model=model, args=training_args, train_dataset= dataset_trans['train'], eval_dataset= dataset_trans['test'], compute_metrics=compute_metrics
)

trainer.train()

  0%|          | 0/6 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss,Accuracy
1,0.338500,0.267363,0.919833
2,0.187800,0.250565,0.926833
3,0.166900,0.213768,0.935167
4,0.112800,0.302637,0.929167
5,0.062300,0.402976,0.928167
6,0.038600,0.411643,0.928500


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-20

TrainOutput(global_step=10500, training_loss=0.17374554724920363, metrics={'train_runtime': 9433.7088, 'train_samples_per_second': 8.904, 'train_steps_per_second': 1.113, 'total_flos': 2.2102122405888e+16, 'train_loss': 0.17374554724920363, 'epoch': 6.0})

In [9]:
! mv /content/test_trainer/checkpoint-4500/ /content/drive/MyDrive/10.9.22-emo_class/training-bert-base-cased/